In [ ]:
import pandas as pd
crashes = pd.read_csv('crashes.csv')
print(crashes.columns.values)

In [ ]:
cleaned_crashes = crashes[['COLLISION_ID','CRASH DATE','CRASH TIME','BOROUGH','ZIP CODE','LATITUDE','LONGITUDE','NUMBER OF PERSONS INJURED']].copy()
cleaned_crashes.to_csv('cleaned_crashes.csv', index=False)

# Vehicles

In [ ]:
import pandas as pd
vehicles = pd.read_csv('vehicles.csv')
print(vehicles)

In [ ]:
cleaned_vehicles = vehicles[['UNIQUE_ID','COLLISION_ID','VEHICLE_ID','VEHICLE_TYPE','VEHICLE_YEAR','VEHICLE_MAKE','VEHICLE_MODEL']].copy()
cleaned_vehicles.to_csv('cleaned_vehicles.csv', index=False)

# Persons

In [ ]:
persons = pd.read_csv('person.csv')

In [ ]:
cleaned_persons = persons[['UNIQUE_ID','PERSON_ID','COLLISION_ID','VEHICLE_ID','PERSON_AGE','PERSON_INJURY','EMOTIONAL_STATUS','PERSON_SEX','PED_ROLE']].copy()
cleaned_persons.to_csv('cleaned_person.csv', index=False)

# Crashes